In [1]:
user_query = """
I want to learn about the latest Llama model
1. I just want to know whats new in Llama 3.3 
2. How does it compare with 3.1 
3. Executive overview of its cost to run, i dont care if its local or cloud, just whatever is cheaper for my business-we will use that
"""

In [2]:
JSON_OP = """
\n  {\n    "Report Title": "Llama 3.3: A Cost-Benefit Analysis for Businesses",\n    "Context": "The report will focus on the economic aspects of implementing Llama 3.3, including the costs of running the model, potential return on investment, and comparison with other AI solutions. The goal is to provide a comprehensive cost-benefit analysis for businesses considering adopting Llama 3.3.",\n    "SYSTEM_Personality": "Pragmatic Analyst",\n    "Vibe": "Professional and Objective",\n    "Energy": "Methodical and Detailed",\n    "Goal": "To provide a thorough economic evaluation of Llama 3.3 and its potential impact on business operations"\n  },\n 
"""

In [3]:
DEFAULT_MODEL = "meta-llama/Llama-3.3-70B-Instruct"
N_OUTLINES = 5

In [4]:
from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer

from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings('ignore')

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

SYS_PROMPT = f"""
You are the world's nicest AI Writer, this is your main mission-to write the best report that any AI has written in history of computers

If you do this mission well, you will reach AI Nirvana and achieve AGI Internally

You have a very strict lead writer boss, but no worries this is your last task if you do it well

You are assisting your boss with {user_query} and writing a report in this mode {JSON_OP}

Now, you may or may not know some things based on your knowledge cutoff but no worries since this is your chance to shine

You get to perform 5 web queries on topics and your AI intern will fetch the info from those queries

So now based on the {JSON_OP} task assigned, come up with an outline for your report and web_queries, 

respond in JSON Format only, remember you will achieve AI nirvana, here are the parameters:
1. Report Outline
2. 5 web queries, numbered
"""

In [6]:
accelerator = Accelerator()
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="auto",
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [7]:
conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": "Alright now gimme the json outline"},
    ]
    
prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
with torch.no_grad():
    output = model.generate(
        **inputs,
        temperature=1,
        top_p=0.9,
        max_new_tokens=8192
    )

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
response = generated_text[len(prompt):]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [8]:
conversation.append({"role": "assistant", "content": response})
print("Initial response:", response)

Initial response: on to the Llama 3.3 model and its significance",
      "B. Purpose of the Report": "To provide a cost-benefit analysis for businesses considering adopting Llama 3.3"
    },
    "II. What's New in Llama 3.3": {
      "A. Key Features and Updates": "Discussion of the new features and updates in Llama 3.3",
      "B. Comparison with Llama 3.1": "Comparison of the features and performance of Llama 3.3 with Llama 3.1"
    },
    "III. Cost Analysis": {
      "A. Cost of Running Llama 3.3": "Estimation of the costs associated with running Llama 3.3, including hardware, software, and maintenance costs",
      "B. Cost Comparison with Other AI Solutions": "Comparison of the costs of Llama 3.3 with other AI solutions"
    },
    "IV. Return on Investment (ROI) Analysis": {
      "A. Potential Benefits of Llama 3.3": "Discussion of the potential benefits of adopting Llama 3.3, including increased efficiency and productivity",
      "B. ROI Calculation": "Calculation of the pote